In [1]:
import numpy as np
import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import datetime

In [2]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2021-10-15 23:24:03.482667: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-15 23:24:03.491886: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-15 23:24:03.492491: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
IMPORT_COUNT = 5000000
TEST_COUNT = 10000

In [4]:
# Generate random seed
#myrand=np.random.randint(1, 99999 + 1)
myrand=71926
np.random.seed(myrand)
tf.random.set_seed(myrand)
print("Random seed is:",myrand)

Random seed is: 71926


In [5]:
RNG_OUTPUT_FILENAME="mersenne_twist_states.txt"
df = np.genfromtxt(RNG_OUTPUT_FILENAME,delimiter='\n',dtype='uint64', max_rows=IMPORT_COUNT)

In [6]:
#calculates how many bits are in the output.
BIT_WIDTH=np.ceil(np.log2(np.amax(df))).astype(int)
# convert the generated numbers to binary sequences
df_as_bits =(df[:,None] & (1 << np.arange(BIT_WIDTH,dtype='uint64')) > 0).astype(int)

In [7]:
#convert the data into inputs and outputs
df_as_frames = np.array([df_as_bits[[X - 624, X - 624 + 1, X- 624 +397, X]] for X in range(624, df_as_bits.shape[0])])

In [8]:
# shuffle the data frames
indicies = np.arange(df_as_frames.shape[0],dtype='uint64')
np.random.shuffle(indicies)
df_as_frames=df_as_frames[indicies]

In [9]:
# convert the data into inputs and outputs
y = df_as_frames[:,-1,:]
X = df_as_frames[:,:-1,]
X = X.reshape([X.shape[0], X.shape[1]*X.shape[2]])[:, 31:]

In [10]:
# Split the data into train and test data
X_train = X[TEST_COUNT:]
X_test = X[:TEST_COUNT]
y_train = y[TEST_COUNT:]
y_test = y[:TEST_COUNT]

In [11]:
def build_model(hp):
    LOSS="binary_crossentropy"
    model = Sequential()
    model.add(Dense(96, activation='relu',input_shape=[X.shape[1]] ))
    model.add(Dense(y.shape[1], activation='sigmoid'))
    
    opt = keras.optimizers.Nadam(
        learning_rate=hp.Float("learning_rate", 10**(-5), 10**(-2),sampling="log"),
        epsilon=hp.Float("epsilon",1e-7,1e-4,sampling="log"),
        beta_1=hp.Float("beta_1",.8,.9999,sampling="reverse_log"),
        beta_2=hp.Float("beta_2",.8,.9999,sampling="reverse_log"),
        )
    model.compile(optimizer=opt, loss=LOSS,metrics=['binary_accuracy'])
    model.summary()
    return model

In [12]:
#define callback functions
stopEarly = tf.keras.callbacks.EarlyStopping(
    monitor='val_binary_accuracy', min_delta=.001, patience=10, verbose=0, mode='auto', restore_best_weights=False
)

log_dir = "tmp/hyperparameters_twisting/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,profile_batch=0)

In [13]:
# extract a short set from the training for hyper parameter tuning
X_train_short= X_train[:1000000]
y_train_short= y_train[:1000000]

In [14]:
%%time
# Tuning the hyper parameters

tuner = kt.tuners.bayesian.BayesianOptimization(build_model,'binary_accuracy',25,project_name="tmp/twisting", seed=myrand)

while tuner.remaining_trials>0:
    try:
        tuner.search(X_train_short, y_train_short,batch_size=2048, epochs=100, validation_data=(X_test,y_test),callbacks=[stopEarly,tensorboard_callback])
    except:
        pass

tuner.results_summary()

Trial 25 Complete [00h 00m 53s]
binary_accuracy: 1.0

Best binary_accuracy So Far: 1.0
Total elapsed time: 00h 25m 54s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in ./tmp/twisting
Showing 10 best trials
Objective(name='binary_accuracy', direction='max')
Trial summary
Hyperparameters:
learning_rate: 0.01
epsilon: 1.0365066146817343e-06
beta_1: 0.8902064726447108
beta_2: 0.9010442893704997
Score: 1.0
Trial summary
Hyperparameters:
learning_rate: 0.01
epsilon: 1.3600174479177251e-05
beta_1: 0.8000000000000002
beta_2: 0.8000000000000002
Score: 1.0
Trial summary
Hyperparameters:
learning_rate: 0.01
epsilon: 4.502635219572926e-07
beta_1: 0.8000000000000002
beta_2: 0.8000000000000002
Score: 1.0
Trial summary
Hyperparameters:
learning_rate: 0.01
epsilon: 3.608238792771209e-06
beta_1: 0.8000000000000002
beta_2: 0.8786715654208318
Score: 1.0
Trial summary
Hyperparameters:
learning_rate: 0.01
epsilon: 4.840144702970955e-06
beta_1: 0.8667004987121238
beta_2: 0.8381878992194518
S

In [15]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
# use the best model for training
model = tuner.hypermodel.build(best_hps)

best_hps.values

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 96)                6336      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                3104      
Total params: 9,440
Trainable params: 9,440
Non-trainable params: 0
_________________________________________________________________


{'learning_rate': 0.01,
 'epsilon': 1.0365066146817343e-06,
 'beta_1': 0.8902064726447108,
 'beta_2': 0.9010442893704997}

In [16]:
def train_model(model, X = X_train, Y=y_train, epochs=100, batch_size=2048,verbose=1, log_dir = "tmp/dense_model/"):
    log_dir += datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,profile_batch=0)
    model.fit(X, Y, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size,callbacks=[tensorboard_callback],verbose=verbose)
    return model

In [17]:
%%time
model = train_model(model, epochs=20,log_dir = "tmp/mt_twisting_model/")

2021-10-15 23:50:38.100127: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2594475520 exceeds 10% of free system memory.
2021-10-15 23:50:40.843615: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2594475520 exceeds 10% of free system memory.


Epoch 1/20
2437/2437 [==============================] - 8s 3ms/step - loss: 0.2740 - binary_accuracy: 0.8152 - val_loss: 0.1648 - val_binary_accuracy: 0.8888
Epoch 2/20
2437/2437 [==============================] - 7s 3ms/step - loss: 0.1092 - binary_accuracy: 0.9245 - val_loss: 0.0630 - val_binary_accuracy: 0.9573
Epoch 3/20
2437/2437 [==============================] - 7s 3ms/step - loss: 0.0246 - binary_accuracy: 0.9864 - val_loss: 2.2062e-06 - val_binary_accuracy: 1.0000
Epoch 4/20
2437/2437 [==============================] - 7s 3ms/step - loss: 5.9490e-07 - binary_accuracy: 1.0000 - val_loss: 1.9736e-07 - val_binary_accuracy: 1.0000
Epoch 5/20
2437/2437 [==============================] - 7s 3ms/step - loss: 1.4672e-07 - binary_accuracy: 1.0000 - val_loss: 1.0430e-07 - val_binary_accuracy: 1.0000
Epoch 6/20
2437/2437 [==============================] - 7s 3ms/step - loss: 8.7978e-08 - binary_accuracy: 1.0000 - val_loss: 7.0819e-08 - val_binary_accuracy: 1.0000
Epoch 7/20
2437/2437 [==

In [18]:
results = model.evaluate(X_test, y_test, batch_size=256)
print("test loss: %f, test acc: %s" % tuple(results))

40/40 [==============================] - 0s 2ms/step - loss: 1.3283e-08 - binary_accuracy: 1.0000
test loss: 0.000000, test acc: 1.0


In [19]:
model.save("mt_twisting_model.h5")